# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: **LoRa**
* Model: **GPT-2**
* Evaluation approach: **Evaluate method with a Hugging Face Trainer**
* Fine-tuning dataset: **IMDB (stanfordnlp/imdb)**

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install -q --no-warn-script-location "datasets==2.15.0"
!pip install -q transformers
!pip install -q torch

In [2]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import numpy as np
import torch

In [3]:
def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    return tokenizer(examples['text'], padding = 'max_length', truncation = True, max_length = 128)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [4]:
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}

for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
model = GPT2ForSequenceClassification.from_pretrained(
    'gpt2',
    num_labels = 2,
    id2label = {0: 'NEGATIVE', 1: 'POSITIVE'},
    label2id = {'NEGATIVE': 0, 'POSITIVE': 1}

)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model.config.pad_token_id = tokenizer.pad_token_id

In [8]:
tokenized_ds = {split: ds[split].map(preprocess_function, batched=True) for split in splits}

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
for param in model.base_model.parameters():
    param.requires_grad = False

In [10]:
training_args = TrainingArguments(
    output_dir = './results',
    learning_rate = 2e-3,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 3,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)


trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds['train'],
    eval_dataset = tokenized_ds['test'],
    tokenizer = tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer, True, 'pt'),
    compute_metrics = compute_metrics
)

In [11]:
eval_results = trainer.evaluate()
print(f'Evaluation Results: {eval_results}')

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Evaluation Results: {'eval_loss': 5.516880035400391, 'eval_accuracy': 0.494, 'eval_runtime': 4.3634, 'eval_samples_per_second': 114.59, 'eval_steps_per_second': 14.438}


In [35]:
print(f"Model Accuracy Before Fine-Tuning: {eval_results['eval_accuracy'] * 100:.2f}%")

Model Accuracy Before Fine-Tuning: 49.40%


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [13]:
!pip install -q peft
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification

In [14]:
train_lora = tokenized_ds['train'].rename_column('label', 'labels')
test_lora = tokenized_ds['test'].rename_column('label', 'labels')

In [15]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.01,
    bias="none",
    fan_in_fan_out=True,
    task_type= 'SEQ_CLS'
)

lora_model = get_peft_model(model, config)

In [16]:
peft_training_args = TrainingArguments(
    output_dir = './lora_results',
    learning_rate = 2e-3,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 3,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

peft_trainer = Trainer(
    model = lora_model,
    args = peft_training_args,
    train_dataset = train_lora,
    eval_dataset = test_lora,
    tokenizer = tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer, True, 'pt'),
    compute_metrics = compute_metrics,
)

peft_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.675717,0.534000
2,No log,0.563470,0.712000
3,No log,0.428823,0.822000


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


TrainOutput(global_step=189, training_loss=0.6764026217990451, metrics={'train_runtime': 39.6425, 'train_samples_per_second': 37.838, 'train_steps_per_second': 4.768, 'total_flos': 98327789568000.0, 'train_loss': 0.6764026217990451, 'epoch': 3.0})

In [17]:
lora_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [18]:
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
peft_eval_results = peft_trainer.evaluate()
print(f'Evaluation Results: {peft_eval_results}')

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Evaluation Results: {'eval_loss': 0.4288225471973419, 'eval_accuracy': 0.822, 'eval_runtime': 4.0122, 'eval_samples_per_second': 124.62, 'eval_steps_per_second': 15.702, 'epoch': 3.0}


In [36]:
print(f"Model Accuracy After Fine-Tuning: {peft_eval_results['eval_accuracy'] * 100:.2f}%")

Model Accuracy After Fine-Tuning: 82.20%
